# Part 2: Mining the web: Web APIs

We hope the preceding exercise was painful: even with tools to process HTML, it is rough downloading raw HTML and trying to extract information from it!

> Can you think of any other reasons why scraping websites for data in this way is not a good idea?

Luckily, many websites provide an application programming interface (API) for querying their data or otherwise accessing their services from your programs. For instance, Twitter provides a web API for gathering tweets, Flickr provides one for gathering image data, and Github for accessing information about repository histories.

These kinds of web APIs are much easier to use than, for instance, the preceding technique which scrapes raw web pages and then has to parse the resulting HTML. Moreover, there are more scalable in the sense that the web servers can transmit structured data in a less verbose form than raw HTML.

As a starting example, here is some code to look at the activity on Github related to the public version of one of our course's materials.

In [13]:
from IPython.display import display, Markdown

import requests
response = requests.get('https://api.github.com/repos/hpcgarage/prymer/events')

headers_highlighted = str(response.headers).replace("'Content-Type': 'application/json;",
                                                    "'Content-Type': '**application/json**;")
display(Markdown(headers_highlighted))

{'date': 'Fri, 02 Oct 2020 22:57:32 GMT', 'server': 'GitHub.com', 'status': '304 Not Modified', 'cache-control': 'public, max-age=60, s-maxage=60', 'vary': 'Accept, Accept-Encoding, Accept, X-Requested-With, Accept-Encoding', 'etag': '"69c2455b4b0d7f5fa3b92792f3094aca621cb1dc177a895812a960917335609e"', 'last-modified': 'Wed, 09 Sep 2020 21:53:02 GMT', 'access-control-expose-headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, Deprecation, Sunset', 'access-control-allow-origin': '*', 'strict-transport-security': 'max-age=31536000; includeSubdomains; preload', 'x-frame-options': 'deny', 'x-content-type-options': 'nosniff', 'x-xss-protection': '1; mode=block', 'referrer-policy': 'origin-when-cross-origin, strict-origin-when-cross-origin', 'content-security-policy': "default-src 'none'", 'content-type': 'application/json; charset=utf-8', 'x-poll-interval': '60', 'x-github-media-type': 'github.v3; format=json', 'content-encoding': 'gzip', 'X-Ratelimit-Limit': '60', 'X-Ratelimit-Remaining': '56', 'X-Ratelimit-Reset': '1601682888', 'X-Ratelimit-Used': '4', 'Accept-Ranges': 'bytes', 'Content-Length': '2551', 'X-GitHub-Request-Id': 'BE34:851F:74186A:8A9A85:5F77B062'}

Note the `Content-Type` of the response:

In [14]:
print(response.headers['Content-Type'])

application/json; charset=utf-8


The response is in JSON format, which is an open format for exchanging semi-structured data. (JSON stands for **J**ava**S**cript **O**bject **N**otation.) JSON is designed to be human-readable and machine-readable, and maps especially well in Python to nested dictionaries. Let's take a look.

> See also [this tutorial](https://www.w3schools.com/js/js_json_intro.asp) for a JSON primer. JSON is among _the_ universal formats for sharing data on the web; see, for instance, https://www.sitepoint.com/10-example-json-files/.

In [15]:
import json
print(type(response.json()))
print(json.dumps(response.json()[:3], sort_keys=True, indent=2))

<class 'list'>
[
  {
    "actor": {
      "avatar_url": "https://avatars.githubusercontent.com/u/64992338?",
      "display_login": "DKillian7",
      "gravatar_id": "",
      "id": 64992338,
      "login": "DKillian7",
      "url": "https://api.github.com/users/DKillian7"
    },
    "created_at": "2020-09-09T21:53:02Z",
    "id": "13465970482",
    "org": {
      "avatar_url": "https://avatars.githubusercontent.com/u/6626190?",
      "gravatar_id": "",
      "id": 6626190,
      "login": "hpcgarage",
      "url": "https://api.github.com/orgs/hpcgarage"
    },
    "payload": {
      "action": "started"
    },
    "public": true,
    "repo": {
      "id": 202786912,
      "name": "hpcgarage/prymer",
      "url": "https://api.github.com/repos/hpcgarage/prymer"
    },
    "type": "WatchEvent"
  },
  {
    "actor": {
      "avatar_url": "https://avatars.githubusercontent.com/u/3359815?",
      "display_login": "snelluri",
      "gravatar_id": "",
      "id": 3359815,
      "login": "snellu

**Exercise 0.** It should be self-evident that the JSON response above consists of a sequence of records, which we will refer to as _events_. Each event is associated with an _actor_. Write some code to extract a dictionary of all actors, where the key is the actor's login and the value is the actor's URL.

In [16]:
import json
def extract_actors (json_github_events):
    """Given JSON records for events in a GitHub repo,
    returns a dictionary of the actors and their URLs.
    """
#     return [{json.dumps(actor['actor']['login']):json.dumps(actor['actor']['url'])} for actor in json_github_events]
#     i=0
    ret={}
    for actor in json_github_events:
        act=json.dumps(actor['actor']['login'])
        act=act.replace('\"','')
        url=json.dumps(actor['actor']['url']).replace('\"','')
        ret[act]=url
    return ret
    
            
extract_actors(response.json())

{'DKillian7': 'https://api.github.com/users/DKillian7',
 'snelluri': 'https://api.github.com/users/snelluri',
 'acollaguazo': 'https://api.github.com/users/acollaguazo',
 'YYCJag': 'https://api.github.com/users/YYCJag',
 'yvemeng': 'https://api.github.com/users/yvemeng',
 'lelelena': 'https://api.github.com/users/lelelena',
 'shuoooooooooo': 'https://api.github.com/users/shuoooooooooo',
 'bduja': 'https://api.github.com/users/bduja'}

In [17]:
actor_urls = extract_actors(response.json ())

for actor, url in actor_urls.items ():
    print ('{}: {}'.format(actor, url))
    assert url == "https://api.github.com/users/{}".format(actor)

DKillian7: https://api.github.com/users/DKillian7
snelluri: https://api.github.com/users/snelluri
acollaguazo: https://api.github.com/users/acollaguazo
YYCJag: https://api.github.com/users/YYCJag
yvemeng: https://api.github.com/users/yvemeng
lelelena: https://api.github.com/users/lelelena
shuoooooooooo: https://api.github.com/users/shuoooooooooo
bduja: https://api.github.com/users/bduja


**Exercise 1.** Write some code that goes to each actor's URL and determines their name. If an actor URL is invalid, that actor should not appear in the output.

In [18]:
import re
def lookup_names (actor_urls):
    """Given a dictionary of (actor, url) pairs, looks up the JSON at
    the URL and extracts the user's name (if any). Returns a new
    dictionary of (actor, name) pairs.
    """
    print(actor_urls)
    ret={}
    for each in actor_urls.keys():
        url=actor_urls[each]
        actors=re.findall('(?<=users\/)(.+?)$',url)
        if len(actors)>0:
            ret[each]=actors[0]
    return ret


That's the end of this notebook. Processing JSON is fairly straightforward, because it maps very naturally to nested dictionaries in Python. You might search the web for other sources of JSON data, including [this one](https://www.yelp.com/dataset/challenge), and do your own processing!